In [ ]:
%load_ext autoreload
%matplotlib inline

import os
from pathlib import Path
from tempfile import TemporaryDirectory

import numpy as np
import nibabel as nb

# cwd = TemporaryDirectory()
# os.chdir(cwd.name)
# print(os.getcwd())

In [ ]:
!wget https://nipy.org/nibabel/_downloads/someones_anatomy.nii.gz

In [ ]:
# Load the example
nii = nb.load('someones_anatomy.nii.gz')
hdr = nii.header.copy()
aff = nii.affine.copy()
data = nii.get_data()

nii.orthoview()

In [ ]:
# Create LAS version
las_aff = aff.copy()
las_aff[0, 0] *= -1.0
las_aff[0, 3] = aff.dot(np.hstack((np.array(nii.shape[:3]) - 1, 1.0)))[0]

las = nb.Nifti1Image(np.flip(data, 0), las_aff, nii.header)
las.to_filename('someones_anatomy_LAS.nii.gz')

# Create LPS version
lps_aff = aff.copy()
lps_aff[0, 0] *= -1.0
lps_aff[1, 1] *= -1.0
lps_aff[:2, 3] = aff.dot(np.hstack((np.array(nii.shape[:3]) - 1, 1.0)))[:2]

lps = nb.Nifti1Image(np.flip(np.flip(data, 0), 1), lps_aff, nii.header)
lps.to_filename('someones_anatomy_LPS.nii.gz')

# Create version not aligned to canonical
R = nb.affines.from_matvec(nb.eulerangles.euler2mat(x=0.09, y=0.001, z=0.001))
new_aff = R.dot(aff)
new = nb.Nifti1Image(data, new_aff, hdr)
new.header.set_qform(new_aff, 1)
new.header.set_sform(new_aff, 1)
new.to_filename('someones_anatomy_rot.nii.gz')

In [ ]:
M = nb.affines.from_matvec(nb.eulerangles.euler2mat(x=0.09, y=0.001, z=0.001), [4.0, 2.0, -1.0])

In [ ]:
xfm = nb.transform.linear.Affine(M)
xfm.matrix

In [ ]:
xfm.reference = nb.load('someones_anatomy_rot.nii.gz')
moved = xfm.resample(new)
moved.to_filename('moved-nb.nii.gz')

In [ ]:
moved.orthoview()

In [ ]:
xfm.to_filename('M.tfm', fmt='itk')
xfm.to_filename('M.fsl', fmt='fsl')
xfm.to_filename('M.afni', fmt='afni')

In [ ]:
!antsApplyTransforms -d 3 -i 'someones_anatomy_rot.nii.gz' -r 'someones_anatomy_rot.nii.gz' -o 'moved-itk.nii.gz' -n 'BSpline' -t 'M.tfm' --float

In [ ]:
nb.load('moved-itk.nii.gz').orthoview()

In [ ]:
!flirt -in someones_anatomy_rot.nii.gz -ref someones_anatomy_rot.nii.gz -out moved-fsl.nii.gz -init M.fsl -applyxfm

In [ ]:
nb.load('moved-fsl.nii.gz').orthoview()

In [ ]:
!3dAllineate -base someones_anatomy_rot.nii.gz -source someones_anatomy_rot.nii.gz -1Dmatrix_apply M.afni -prefix moved-afni -final cubic
!3dAFNItoNIFTI moved-afni+orig
!gzip moved-afni.nii

In [ ]:
nb.load('moved-afni.nii.gz').orthoview()